In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
filename = 'results2.csv'

In [ ]:
test_dir = 'tests/test2'

In [ ]:
''' Merge csv files '''

dlist = []
dirs = [d for d in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir,d))]
for d in dirs:
	d = os.path.abspath(test_dir+'/'+d)
	dlist += [os.path.join(d,file) for file in os.listdir(d) if file.endswith(".csv")]

In [ ]:
df = pd.read_csv(dlist[0], skipinitialspace=True)
for x in range(1,len(dlist)):
    df = df.append(pd.read_csv(dlist[x], skipinitialspace=True))
df.to_csv(filename,index=False)

In [ ]:
def autolabel(rects):
	for rect in rects:
		height = rect.get_height()
		ax.annotate('{}'.format(height),
					xy=(rect.get_x() + rect.get_width() / 2, height),
					xytext=(0, 3),  # 3 points vertical offset
					textcoords="offset points",
					ha='center', va='bottom')

In [ ]:
''' No. of optimised structures per method (diff no. of total structures) '''
''' Normalised to per cent '''

ran_succ_list = []
rat_succ_list = []
totals = {}


df = pd.read_csv(filename, skipinitialspace=True)
labels = list(df['method'].unique())
dicts = {}

for method in labels:
	random = df[df['structure'].str.contains(
		"rat") == False].loc[df['method'] == method]
	rattled = df[df['structure'].str.contains(
		"rat")].loc[df['method'] == method]

	ran_succ = len([b for b in random['opt_succ'] if b])
	rat_succ = len([b for b in rattled['opt_succ'] if b])

	ran_perc = 0
	if len(random):
		ran_perc = ran_succ*100/len(random)
	rat_perc = 0
	if len(rattled):
		rat_perc = rat_succ*100/len(rattled)

	ran_succ_list.append(ran_perc)
	rat_succ_list.append(rat_perc)

''' Plot '''
    
y = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()

rects1 = ax.barh(y - width/2, ran_succ_list, width, label='Random') # x% per init category
rects2 = ax.barh(y + width/2, rat_succ_list, width, label='Rattled')

# Add some text for labels, title and custom x-axis tick labels, etc.
for i, v in enumerate(ran_succ_list):
	ax.text(v + 1, i - width/2, str(v))

for i, v in enumerate(rat_succ_list):
	ax.text(v + 1, i + width/2, str(v))

ax.set_title('Successfully optimised structures')
ax.set_xlim(0,110)
ax.set_yticks(y)
ax.set_yticklabels(labels)
ax.invert_yaxis()  # labels read top-to-bottom
ax.legend()

fig.tight_layout()

plt.savefig('OptSucc.pdf', format='pdf')
plt.show()

In [ ]:
''' Find stdev map files '''

map_list = []
for d in dirs:
	d = os.path.abspath(test_dir+'/'+d)
	map_list += [os.path.join(d,file) for file in os.listdir(d) if file == 'map_files.txt']

In [ ]:
''' Make a stdev column '''

df = pd.read_csv(filename, skipinitialspace=True)
df['stdev'] = pd.Series(np.zeros(len(df['structure'])), index=df.index)

In [ ]:
for file in map_list:
    method = file.split('/')[-2]
    for line in open(file,'r'):
        if 'rat_structure' in line:
            stdev = float(line.split(':')[-2].split('/')[-2][3:])
            struct = line.split(':')[-1].lstrip(' ').rstrip('\n')
            mask = (df['method'] == method) & (df['structure'] == struct)
            df.loc[mask,'stdev'] = stdev

In [ ]:
''' Optimisation time per method (diff no. of total structures) '''

FONTSIZE = 12

methods = list(df['method'].unique())
maps = list(df['method'])
ran_succ = df[df['structure'].str.contains(
		"rat") == False].loc[df['opt_succ'] == True] # find successful optimisations
rat_succ = df[df['structure'].str.contains(
		"rat") == True].loc[df['opt_succ'] == True] 

fig, axs = plt.subplots(2,figsize=(15,15))
colors = {'blue' : methods[0], 'orange' : methods[1], 'green' : methods[2]}
clabels = ['blue', 'orange', 'green']

# Random

for color in clabels:
    x = ran_succ.loc[df['method'] == colors[color]]['structure']
    y = ran_succ.loc[df['method'] == colors[color]]['opt_time']
    axs[0].scatter(x, y, c='tab:'+color, label=colors[color],
               alpha=0.7)

# Rattled    

slabels = []
for color in clabels:
    x = rat_succ.loc[df['method'] == colors[color]]['structure']
    y = rat_succ.loc[df['method'] == colors[color]]['opt_time']
    sizes = rat_succ.loc[df['method'] == colors[color]]['stdev']
    slabels += [s for s in sizes.unique() if s not in slabels]
    scatter = axs[1].scatter(x, y, s=sizes*1000, c='tab:'+color,
                            alpha=0.7)

start0, end0 = axs[0].get_ylim()
start1, end1 = axs[1].get_ylim()    

end = max(end0, end1)+1

axs[0].legend(loc="upper left")
axs[0].set_xticks([])
axs[0].set_xlabel('Optimised Structures')
axs[1].set_ylabel('Time (sec)')
axs[0].yaxis.set_ticks(np.arange(0, end))
axs[0].set_title('Random Initialisation', fontsize=FONTSIZE)

h = [plt.scatter([],[], s=i*1000, c="gray") for i in slabels]
leg = plt.legend(handles=h, labels=slabels,loc="upper right", title="Standard Deviation")
axs[1].add_artist(leg)

h = [plt.plot([],[], color='tab:'+c, alpha=0.7, marker="o", ls="")[0] for c in colors]
leg = plt.legend(handles=h, labels=[colors[c] for c in colors],loc="upper left")
axs[1].add_artist(leg)

axs[1].set_xlabel('Optimised Structures')
axs[1].set_ylabel('Time (sec)')
axs[1].yaxis.set_ticks(np.arange(0, end))
axs[1].set_title('Rattled Initialisation', fontsize=FONTSIZE)
axs[1].set_xticks([])

fig.tight_layout()

plt.savefig('OptTime.pdf', format='pdf')
plt.show()

In [ ]:
''' Variation of Optimisation time '''

methods = list(df['method'].unique())
total_succ = df[df['opt_succ'] == True] # times of successful opt/ions
total_succ = total_succ[['method','opt_time']]

# successful optimisations per init
ran_succ = df[df['structure'].str.contains(
		"rat") == False].loc[df['opt_succ'] == True][['method','opt_time']] 
rat_succ = df[df['structure'].str.contains(
		"rat") == True].loc[df['opt_succ'] == True][['method','opt_time']]

# Total
means = total_succ.groupby(['method']).mean()
stdevs = total_succ.groupby(['method']).apply(np.std)

ms = means.join(stdevs, lsuffix='_m', rsuffix='_std')

# Random
ran_means = ran_succ.groupby(['method']).mean()
ran_stdevs = ran_succ.groupby(['method']).apply(np.std)

ran_ms = ran_means.join(ran_stdevs, lsuffix='_m', rsuffix='_std')
ms_all = ms.join(ran_ms, rsuffix='_ran')

# Rattled
rat_means = rat_succ.groupby(['method']).mean()
rat_stdevs = rat_succ.groupby(['method']).apply(np.std)

rat_ms = rat_means.join(rat_stdevs, lsuffix='_m', rsuffix='_std')
ms_all = ms_all.join(rat_ms, rsuffix='_rat')

In [ ]:
fig, ax = plt.subplots()

ind = np.arange(3)  # the x locations for the groups
width = 0.3  # the width of the bars

ax.errorbar(ind - width/3, ms_all['opt_time_m_ran'], ms_all['opt_time_std_ran'], 
            linestyle='None', marker='o', ecolor='red',mfc='red',
            mec='red', label='random')
ax.errorbar(ind, ms_all['opt_time_m'], ms_all['opt_time_std'], linestyle='None', 
            marker='o', ecolor='blue', mfc='blue',
            mec='blue', label='all')
ax.errorbar(ind + width/3, ms_all['opt_time_m_rat'], ms_all['opt_time_std_rat'], 
            linestyle='None', marker='o', ecolor='orange', mfc='orange',
            mec='orange', label='rattled')

ax.set_xticks(ind)
ax.set_xticklabels([row for row in ms_all.index])
ax.set_title('Optimisation Time Deviation')
ax.set_ylabel('Time (sec)')
ax.legend()

fig.tight_layout()
plt.savefig('TimeDev.pdf', format='pdf')
plt.show()

In [ ]:
''' Optimised energy  '''

FONTSIZE = 12

methods = list(df['method'].unique())
maps = list(df['method'])
ran_succ = df[df['structure'].str.contains(
		"rat") == False].loc[df['opt_succ'] == True] # find successful optimisations
rat_succ = df[df['structure'].str.contains(
		"rat") == True].loc[df['opt_succ'] == True] 

fig, axs = plt.subplots(2,figsize=(15,15))
colors = {'blue' : methods[0], 'orange' : methods[1], 'green' : methods[2]}
clabels = ['blue', 'orange', 'green']

# Random

for color in clabels:
    x = ran_succ.loc[df['method'] == colors[color]]['structure']
    y = ran_succ.loc[df['method'] == colors[color]]['energy']
    axs[0].scatter(x, y, c='tab:'+color, label=colors[color],
               alpha=0.7)
    
    
# Rattled    

slabels = []
for color in clabels:
    x = rat_succ.loc[df['method'] == colors[color]]['structure']
    y = rat_succ.loc[df['method'] == colors[color]]['energy']
    sizes = rat_succ.loc[df['method'] == colors[color]]['stdev']
    slabels += [s for s in sizes.unique() if s not in slabels]
    scatter = axs[1].scatter(x, y, s=sizes*1000, c='tab:'+color,
                            alpha=0.7)

start0, end0 = axs[0].get_ylim()
start1, end1 = axs[1].get_ylim()    

start = min(start0, start1)
end = max(end0, end1)+1

axs[0].legend(loc="upper left")
axs[0].set_xticks([])
axs[0].set_xlabel('Optimised Structures')
axs[0].set_ylabel('Energy')
axs[0].yaxis.set_ticks(np.arange(start, end))
axs[0].set_title('Random Initialisation', fontsize=FONTSIZE)

h = [plt.scatter([],[], s=i*1000, c="gray") for i in slabels]
leg = plt.legend(handles=h, labels=slabels,loc="upper right", title="Standard Deviation")
axs[1].add_artist(leg)

h = [plt.plot([],[], color='tab:'+c, alpha=0.7, marker="o", ls="")[0] for c in colors]
leg = plt.legend(handles=h, labels=[colors[c] for c in colors],loc="upper left")
axs[1].add_artist(leg)

axs[1].set_xlabel('Optimised Structures')
axs[1].set_ylabel('Energy')
axs[1].yaxis.set_ticks(np.arange(start, end))
axs[1].set_title('Rattled Initialisation', fontsize=FONTSIZE)
axs[1].set_xticks([])

fig.tight_layout()

plt.savefig('OptEnergy.pdf', format='pdf')
plt.show()